In [2]:
import tensorflow as tf
import pandas as pd
import re
import glob
import cv2

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_ApKoW4T.csv")

In [4]:
test.head()

,image
0,1007700.jpg
1,1011369.jpg
2,1051155.jpg
3,1062001.jpg
4,1069397.jpg


In [5]:
train['image'].head()
train['category'][2]

2

In [6]:
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
import numpy as np

class ImageToVec():
    
    def __init__(self):
        '''
        Constructor for SenttoVec to convert image into fixed length vectors with length 4096
        '''
        self.model = self.create_model()
    
    def get_vector(self,image_path):
        '''
        This method is used to pre-process the input image and return corresponding vector.
        Input: image_path: path where image is located.
        Returns: vector representation of the image.
        '''
        #Loading Image
        img = image.load_img(image_path, target_size=(250, 250))
        img_data = image.img_to_array(img)
        #Adding batch size dimension to the image shape to able to forward pass thorugh model
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        im_vec = self.model.predict(img_data)
        #print(im_vec.shape)
        #Reshaping 4 dimensional image vector into single dimensional vector
        im_vec = im_vec.squeeze()
        im_vec = im_vec.reshape(im_vec.shape[0]*im_vec.shape[1]*im_vec.shape[2])
        return im_vec
        
    def create_model(self):
        '''
        This method is used to build model with pretrained ResNet50.
        Returns: model
        '''
        model = ResNet50(weights='imagenet', include_top=False)
        return model

Using TensorFlow backend.


In [7]:
img_vec = ImageToVec()

C:\Users\JanU\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
DATASET_LOCATION = "images"
hcr = {"data":[], "labels":[], "cls":[]}

images = hcr.get("data")[3000:3009]
cls_true = hcr.get("cls")[3000:3009]
len(images)
plot_images(images=images, cls_true=cls_true)
ax.imshow(images[i], cmap="binary")



In [8]:
len(train)
DATASET_LOCATION = "images"
hcr = {"data":[], "labels":[], "cls":[]}
"""
for i in range (5):
    img_name = train['image'][i]
    img_label= train['category'][i]
    img_path = DATASET_LOCATION + "/" + img_name
    print (img_path)
    Image_show(img_path)
"""
for i in range (5):
    img_name = train['image'][i]
    img_label= train['category'][i]
    img_path = DATASET_LOCATION + "/" + img_name
    print (img_path)
    Image_show(img_path)

images/2823080.jpg


NameError: name 'Image_show' is not defined

In [9]:
from matplotlib import pyplot as plt

def Image_show(img_path):
    img = cv2.imread(img_path)
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.imshow(img)
    plt.title('my picture')
    plt.show()

In [10]:
NUM_CLASSES = 5

# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [11]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

model = Sequential()
model.add(ResNet50(weights='imagenet', include_top=False, pooling = RESNET50_POOLING_AVERAGE))
# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
#model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = resnet_weights_path))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


In [13]:
from tensorflow.python.keras import optimizers

sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [14]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

image_size = 250
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = data_generator.flow_from_directory(
        'images',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        'Validation',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical')

Found 6252 images belonging to 5 classes.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Validation'

In [15]:
(BATCH_SIZE_TRAINING, len(train_generator))  #BATCH_SIZE_VALIDATION, len(validation_generator)

(100, 63)

In [16]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [17]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
)


Epoch 1/10
10/10 [==============================] - 1825s 183s/step - loss: 1.2909 - acc: 0.4750
Epoch 2/10
10/10 [==============================] - 1775s 177s/step - loss: 0.5441 - acc: 0.8010
Epoch 3/10
10/10 [==============================] - 1941s 194s/step - loss: 0.4798 - acc: 0.8080
Epoch 4/10
10/10 [==============================] - 1575s 157s/step - loss: 0.5034 - acc: 0.8110
Epoch 5/10
10/10 [==============================] - 1626s 163s/step - loss: 0.4301 - acc: 0.8420
Epoch 6/10
10/10 [==============================] - 2012s 201s/step - loss: 0.4072 - acc: 0.8590
Epoch 7/10
10/10 [==============================] - 2636s 264s/step - loss: 0.3680 - acc: 0.8533
Epoch 8/10
10/10 [==============================] - 2772s 277s/step - loss: 0.3191 - acc: 0.8770
Epoch 9/10
10/10 [==============================] - 1782s 178s/step - loss: 0.3356 - acc: 0.8710
Epoch 10/10
10/10 [==============================] - 1598s 160s/step - loss: 0.3668 - acc: 0.8520


In [13]:
import shutil
import os

In [22]:
for i in range (len(train)): 
    img_name = train['image'][i]
    img_label= train['category'][i]
    img_path = DATASET_LOCATION + "/" + img_name
    #print (img_path)
    if img_label == 1:
        dst = DATASET_LOCATION + "/" + "1"
        if not os.path.exists(DATASET_LOCATION + "/" +"1"):
            os.makedirs(DATASET_LOCATION + "/" +"1")
            shutil.copy(img_path, DATASET_LOCATION + "/1")
            #dst = DATASET_LOCATION + "/" + "1"
            #os.system ("mv"+ " " + img_path + " " + dst)
        else:
            #os.system ("mv"+ " " + img_path + " " + dst)
            shutil.copy(img_path, DATASET_LOCATION + "/1")

    elif img_label == 2:
        dst = DATASET_LOCATION + "/" + "2"
        if not os.path.exists(DATASET_LOCATION + "/" +"2"):
            os.makedirs(DATASET_LOCATION + "/" +"2")
            shutil.copy(img_path, DATASET_LOCATION + "/2")
        else:
            shutil.copy(img_path, DATASET_LOCATION + "/2")
    elif img_label == 3:
        dst = DATASET_LOCATION + "/" + "3"
        if not os.path.exists(DATASET_LOCATION + "/" +"3"):
            os.makedirs(DATASET_LOCATION + "/" +"3")
            shutil.copy(img_path, DATASET_LOCATION + "/3")
        else:
            shutil.copy(img_path, DATASET_LOCATION + "/3")
    elif img_label == 4:
        dst = DATASET_LOCATION + "/" + "4"
        if not os.path.exists(DATASET_LOCATION + "/" +"4"):
            os.makedirs(DATASET_LOCATION + "/" +"4")
            shutil.copy(img_path, DATASET_LOCATION + "/4")
        else:
            shutil.copy(img_path, DATASET_LOCATION + "/4")
    elif img_label == 5:
        dst = DATASET_LOCATION + "/" + "5"
        if not os.path.exists(DATASET_LOCATION + "/" +"5"):
            os.makedirs(DATASET_LOCATION + "/" +"5")
            shutil.copy(img_path, DATASET_LOCATION + "/5")
        else:
            shutil.copy(img_path, DATASET_LOCATION + "/5")

print("Total Files got finished:", i )

images/2823080.jpg
images/2870024.jpg
images/2662125.jpg
images/2900420.jpg
images/2804883.jpg
images/621252.jpg
images/2833467.jpg
images/2843780.jpg
images/2859567.jpg
images/2896557.jpg
images/2847618.jpg
images/2874678.jpg
images/2903279.jpg
images/2892401.jpg
images/2891158.jpg
images/2896519.jpg
images/2870078.jpg
images/2845743.jpg
images/2692646.jpg
images/2839660.jpg
images/2868091.jpg
images/2812909.jpg
images/2862039.jpg
images/2890672.jpg
images/2874072.jpg
images/2785048.jpg
images/683110.jpg
images/2877843.jpg
images/2872097.jpg
images/2710618.jpg
images/1414706.jpg
images/2839562.jpg
images/2896605.jpg
images/2845690.jpg
images/947703.jpg
images/2816830.jpg
images/2845772.jpg
images/2904591.jpg
images/2880224.jpg
images/2653226.jpg
images/2809869.jpg
images/2819017.jpg
images/2849471.jpg
images/2819587.jpg
images/2843653.jpg
images/2818966.jpg
images/2408177.jpg
images/2899497.jpg
images/2560213.jpg
images/2849863.jpg
images/2810838.jpg
images/2838423.jpg
images/2100454.

images/2788351.jpg
images/1887777.jpg
images/2880265.jpg
images/2730606.jpg
images/2816832.jpg
images/2808832.jpg
images/2904582.jpg
images/2812800.jpg
images/2859716.jpg
images/2796474.jpg
images/2898609.jpg
images/2704899.jpg
images/2903004.jpg
images/2792347.jpg
images/2833486.jpg
images/2896516.jpg
images/2790323.jpg
images/2839507.jpg
images/741153.jpg
images/1187990.jpg
images/2776116.jpg
images/2778107.jpg
images/659008.jpg
images/2885047.jpg
images/2847617.jpg
images/2122710.jpg
images/2818978.jpg
images/2871877.jpg
images/2853911.jpg
images/2796404.jpg
images/2898537.jpg
images/2849884.jpg
images/2780206.jpg
images/2788306.jpg
images/2894438.jpg
images/2861842.jpg
images/2895782.jpg
images/2147838.jpg
images/2890484.jpg
images/2851891.jpg
images/2857689.jpg
images/2796424.jpg
images/2866156.jpg
images/2858393.jpg
images/2790315.jpg
images/2894545.jpg
images/2789483.jpg
images/2849916.jpg
images/2812798.jpg
images/2890432.jpg
images/2900695.jpg
images/2874362.jpg
images/2887163

images/2849908.jpg
images/2812919.jpg
images/2823049.jpg
images/2860036.jpg
images/2796473.jpg
images/2900547.jpg
images/2886862.jpg
images/2907016.jpg
images/2870088.jpg
images/2818939.jpg
images/2847647.jpg
images/2896460.jpg
images/2892462.jpg
images/2861990.jpg
images/2806788.jpg
images/2778067.jpg
images/2502524.jpg
images/2825281.jpg
images/2788268.jpg
images/2837561.jpg
images/2823225.jpg
images/2835433.jpg
images/2136367.jpg
images/2788195.jpg
images/2829421.jpg
images/2860050.jpg
images/2835377.jpg
images/2898554.jpg
images/2902612.jpg
images/2857335.jpg
images/2851805.jpg
images/2097227.jpg
images/2851972.jpg
images/2869216.jpg
images/2894167.jpg
images/2531139.jpg
images/2873033.jpg
images/2840924.jpg
images/2894435.jpg
images/2864149.jpg
images/2827194.jpg
images/2366014.jpg
images/2837646.jpg
images/2788235.jpg
images/2879987.jpg
images/718949.jpg
images/2892549.jpg
images/901189.jpg
images/2879918.jpg
images/2829387.jpg
images/2864058.jpg
images/2844068.jpg
images/1375274

images/2888410.jpg
images/2896567.jpg
images/2850397.jpg
images/2902009.jpg
images/2872999.jpg
images/2829372.jpg
images/2893284.jpg
images/2837959.jpg
images/2784281.jpg
images/643083.jpg
images/2792287.jpg
images/2878413.jpg
images/2847691.jpg
images/2848729.jpg
images/2720150.jpg
images/2255446.jpg
images/942931.jpg
images/2786200.jpg
images/2798445.jpg
images/2888477.jpg
images/2816858.jpg
images/2799304.jpg
images/2900631.jpg
images/2818982.jpg
images/2880259.jpg
images/2798501.jpg
images/2904467.jpg
images/2870032.jpg
images/2814380.jpg
images/2780126.jpg
images/2850554.jpg
images/2821121.jpg
images/2839095.jpg
images/702096.jpg
images/1885516.jpg
images/2856014.jpg
images/2868019.jpg
images/2909644.jpg
images/633184.jpg
images/1384029.jpg
images/2861839.jpg
images/2902689.jpg
images/2796402.jpg
images/2804681.jpg
images/2732658.jpg
images/2816886.jpg
images/2788352.jpg
images/2860115.jpg
images/2847708.jpg
images/2902831.jpg
images/2808049.jpg
images/2782123.jpg
images/2876429.j

images/2794454.jpg
images/2458876.jpg
images/2864878.jpg
images/2808775.jpg
images/2872905.jpg
images/2778108.jpg
images/2507826.jpg
images/2784256.jpg
images/2903476.jpg
images/1653912.jpg
images/2890400.jpg
images/792103.jpg
images/2864118.jpg
images/2744699.jpg
images/2830833.jpg
images/2892732.jpg
images/2892962.jpg
images/2843692.jpg
images/2814888.jpg
images/2880782.jpg
images/2807306.jpg
images/1721196.jpg
images/2880307.jpg
images/2823055.jpg
images/2904737.jpg
images/2890263.jpg
images/1884644.jpg
images/2849824.jpg
images/2711898.jpg
images/2840989.jpg
images/2905604.jpg
images/2870035.jpg
images/2874191.jpg
images/2829394.jpg
images/2833461.jpg
images/1905585.jpg
images/2890393.jpg
images/1872456.jpg
images/2868052.jpg
images/2847684.jpg
images/2856065.jpg
images/2897051.jpg
images/2870140.jpg
images/2772024.jpg
images/2892457.jpg
images/2896503.jpg
images/2837511.jpg
images/2672269.jpg
images/2798486.jpg
images/2829320.jpg
images/1737608.jpg
images/2812792.jpg
images/290522

...

images/961864.jpg
images/2790258.jpg
images/2898547.jpg
images/2786153.jpg
images/2784217.jpg
images/2835416.jpg
images/2903461.jpg
images/2806668.jpg
images/2547522.jpg
images/2876638.jpg
images/2870132.jpg
images/2902568.jpg
images/2865611.jpg
images/2882019.jpg
images/2882710.jpg
images/1162668.jpg
images/2875075.jpg
images/2843628.jpg
images/1962341.jpg
images/1651137.jpg
images/2819039.jpg
images/2825195.jpg
images/1798575.jpg
images/2851823.jpg
images/2773598.jpg
images/953310.jpg
images/2872590.jpg
images/728055.jpg
images/2638485.jpg
images/2656142.jpg
images/2894482.jpg
images/2107324.jpg
images/2798450.jpg
images/2851832.jpg
images/2864086.jpg
images/2784287.jpg
images/2853331.jpg
images/2876129.jpg
images/670844.jpg
images/689834.jpg
images/2898557.jpg
images/2701955.jpg
images/2837550.jpg
images/1210069.jpg
images/2880919.jpg
images/2874112.jpg
images/1001804.jpg
images/2868105.jpg
images/2900557.jpg
images/2565518.jpg
images/2817149.jpg
images/2902676.jpg
images/2864048.jp

images/2849790.jpg
images/2857974.jpg
images/1333613.jpg
images/2819003.jpg
images/2806746.jpg
images/2860146.jpg
images/2870010.jpg
images/2845134.jpg
images/2876156.jpg
images/1003035.jpg
images/1489274.jpg
images/2800581.jpg
images/2843729.jpg
images/2796394.jpg
images/2891442.jpg
images/2898530.jpg
images/2844568.jpg
images/2802601.jpg
images/2864128.jpg
images/2854320.jpg
images/2876181.jpg
images/2842885.jpg
images/2083310.jpg
images/2843663.jpg
images/2897935.jpg
images/2863890.jpg
images/1809796.jpg
images/2863837.jpg
images/2841631.jpg
images/2839580.jpg
images/2865797.jpg
images/2837577.jpg
images/2834292.jpg
images/1126799.jpg
images/2890522.jpg
images/2800419.jpg
images/2816916.jpg
images/2807377.jpg
images/2839480.jpg
images/1010906.jpg
images/2858120.jpg
images/2837736.jpg
images/2823297.jpg
images/1008439.jpg
images/2860059.jpg
images/2808761.jpg
images/2838584.jpg
images/633207.jpg
images/2841568.jpg
images/2882288.jpg
images/2784220.jpg
images/2827276.jpg
images/278000

images/2882267.jpg
images/2894469.jpg
images/1810512.jpg
images/2865703.jpg
images/1979552.jpg
images/2888509.jpg
images/2780207.jpg
images/2810807.jpg
images/2852950.jpg
images/2788324.jpg
images/2109948.jpg
images/2798373.jpg
images/2825164.jpg
images/2808819.jpg
images/2891683.jpg
images/2366326.jpg
images/2874117.jpg
images/2888361.jpg
images/2798378.jpg
images/2878496.jpg
images/2890834.jpg
images/2886419.jpg
images/2840840.jpg
images/2885397.jpg
images/2890364.jpg
images/1431400.jpg
images/2648192.jpg
images/2857691.jpg
images/2644051.jpg
images/2816904.jpg
images/2790298.jpg
images/2837858.jpg
images/2868143.jpg
images/2870093.jpg
images/2812926.jpg
images/1392030.jpg
images/2890451.jpg
images/2896508.jpg
images/619245.jpg
images/2764708.jpg
images/796519.jpg
images/2896504.jpg
images/2798413.jpg
images/2896490.jpg
images/2837798.jpg
images/2897575.jpg
images/674836.jpg
images/2804767.jpg
images/2603561.jpg
images/2878236.jpg
images/2862104.jpg
images/2784254.jpg
images/2862626.

images/2798403.jpg
images/2698727.jpg
images/2814817.jpg
images/2901730.jpg
images/2864030.jpg
images/2831425.jpg
images/2833400.jpg
images/2804496.jpg
images/2788309.jpg
images/2874102.jpg
images/1892834.jpg
images/2007300.jpg
images/1812187.jpg
images/1016682.jpg
images/2778113.jpg
images/2806695.jpg
images/2839570.jpg
images/2711117.jpg
images/2816964.jpg
images/2829277.jpg
images/2875610.jpg
images/2871350.jpg
images/2876805.jpg
images/2831320.jpg
images/2810873.jpg
images/2878190.jpg
images/2648109.jpg
images/2697217.jpg
images/2851801.jpg
images/2902581.jpg
images/2792413.jpg
images/2788232.jpg
images/2893621.jpg
images/643865.jpg
images/2782261.jpg
images/2845704.jpg
images/2718356.jpg
images/2460578.jpg
images/2839533.jpg
images/2884286.jpg
images/1141995.jpg
images/2902695.jpg
images/2298068.jpg
images/2890485.jpg
images/2810820.jpg
images/2904592.jpg
images/2868088.jpg
images/2800428.jpg
images/2812757.jpg
images/2713759.jpg
images/2790253.jpg
images/2841579.jpg
images/287615

images/2843632.jpg
images/2195382.jpg
images/1011813.jpg
images/299806.jpg
images/2899954.jpg
images/2809020.jpg
images/2404333.jpg
images/2806611.jpg
images/2892444.jpg
images/2841735.jpg
images/2892412.jpg
images/2864091.jpg
images/2812857.jpg
images/2860033.jpg
images/1593688.jpg
images/2817602.jpg
images/2810799.jpg
images/1106366.jpg
images/2886161.jpg
images/2802519.jpg
images/2853868.jpg
images/2863324.jpg
images/2867453.jpg
images/1490746.jpg
images/2751844.jpg
images/2846390.jpg
images/2810863.jpg
images/1318660.jpg
images/2792356.jpg
images/832621.jpg
images/2840591.jpg
images/2788189.jpg
images/2816947.jpg
images/2859196.jpg
images/2901519.jpg
images/2829297.jpg
images/2666436.jpg
images/2780676.jpg
images/2890411.jpg
images/2816854.jpg
images/2856932.jpg
images/2788794.jpg
images/2902691.jpg
images/2800502.jpg
images/676641.jpg
images/2804725.jpg
images/1884634.jpg
images/2829439.jpg
images/2780214.jpg
images/2806648.jpg
images/2833416.jpg
images/2816859.jpg
images/2786248.